In [1]:
library(caret, quiet = TRUE)
library(base64enc)
library(httr, quiet = TRUE)

library(mlbench)


Attaching package: ‘httr’

The following object is masked from ‘package:caret’:

    progress



# Build a Model

In [2]:

## multiclass classification in iris dataset:

data(BostonHousing)
BostonHousing$chas = as.numeric(BostonHousing$chas)

set.seed(1960)

dataset = BostonHousing[, -14] 

create_model  =  function() {

    model <- train(medv ~ ., data = BostonHousing, method = "glmnet")    

    return(model)
}


In [3]:
model = create_model()
# cat(model$feature_names)
# print(model)

In [4]:
pred_labels <- predict(model, BostonHousing[, -14] , type="raw")
df = data.frame(BostonHousing[,14])
names(df) = c("medv")
df$Estimator = pred_labels
df$Error = df$Estimator - df$medv
MAPE = mean(abs(df$Error / df$medv))
summary(df)
MAPE

      medv         Estimator          Error         
 Min.   : 5.00   Min.   :-4.209   Min.   :-26.3365  
 1st Qu.:17.02   1st Qu.:17.490   1st Qu.: -1.7354  
 Median :21.20   Median :22.167   Median :  0.5003  
 Mean   :22.53   Mean   :22.533   Mean   :  0.0000  
 3rd Qu.:25.00   3rd Qu.:27.423   3rd Qu.:  2.7089  
 Max.   :50.00   Max.   :44.596   Max.   : 15.6200  

[1] 0.1638929

# SQL Code Generation

In [5]:

test_ws_sql_gen = function(mod) {
    WS_URL = "https://sklearn2sql.herokuapp.com/model"
    WS_URL = "http://localhost:1888/model"
    model_serialized <- serialize(mod, NULL)
    b64_data = base64encode(model_serialized)
    data = list(Name = "xgboost_test_model", SerializedModel = b64_data , SQLDialect = "postgresql" , Mode="caret")
    r = POST(WS_URL, body = data, encode = "json")
    # print(r)
    content = content(r)
    # print(content)
    lSQL = content$model$SQLGenrationResult[[1]]$SQL # content["model"]["SQLGenrationResult"][0]["SQL"]
    return(lSQL);
}

In [6]:
lModelSQL = test_ws_sql_gen(model)
cat(lModelSQL)


WITH linear_input AS 
(SELECT "ADS"."KEY" AS "KEY", CAST("ADS"."Feature_0" AS FLOAT) AS "Feature_0", CAST("ADS"."Feature_1" AS FLOAT) AS "Feature_1", CAST("ADS"."Feature_2" AS FLOAT) AS "Feature_2", CAST("ADS"."Feature_3" AS FLOAT) AS "Feature_3", CAST("ADS"."Feature_4" AS FLOAT) AS "Feature_4", CAST("ADS"."Feature_5" AS FLOAT) AS "Feature_5", CAST("ADS"."Feature_6" AS FLOAT) AS "Feature_6", CAST("ADS"."Feature_7" AS FLOAT) AS "Feature_7", CAST("ADS"."Feature_8" AS FLOAT) AS "Feature_8", CAST("ADS"."Feature_9" AS FLOAT) AS "Feature_9", CAST("ADS"."Feature_10" AS FLOAT) AS "Feature_10", CAST("ADS"."Feature_11" AS FLOAT) AS "Feature_11", CAST("ADS"."Feature_12" AS FLOAT) AS "Feature_12" 
FROM "INPUT_DATA" AS "ADS"), 
linear_model_cte AS 
(SELECT linear_input."KEY" AS "KEY", -0.1060050798535143 * linear_input."Feature_0" + 0.04509271581120177 * linear_input."Feature_1" + 0.01233872269065821 * linear_input."Feature_2" + 2.707062364632181 * linear_input."Feature_3" + -17.2938063905899 * lin

# Execute the SQL Code

In [7]:
library(RODBC)
conn = odbcConnect("pgsql", uid="db", pwd="db", case="nochange")
odbcSetAutoCommit(conn , autoCommit = TRUE)

[1] 0

In [8]:
df_sql = dataset
names(df_sql) = sprintf("Feature_%d",0:(ncol(df_sql)-1))
df_sql$KEY = seq.int(nrow(dataset))

sqlDrop(conn , "INPUT_DATA" , errors = FALSE)
sqlSave(conn, df_sql, tablename = "INPUT_DATA", verbose = FALSE)

# df_sql

In [9]:
colnames(df_sql)
# odbcGetInfo(conn)
# sqlTables(conn)

[1] "Feature_0"  "Feature_1"  "Feature_2"  "Feature_3"  "Feature_4" 
 [6] "Feature_5"  "Feature_6"  "Feature_7"  "Feature_8"  "Feature_9" 
[11] "Feature_10" "Feature_11" "Feature_12" "KEY"

In [10]:
df_sql_out = sqlQuery(conn, lModelSQL)
head(df_sql_out[order(df_sql_out$KEY),])

KEY,Estimator
1,30.06640
2,25.01567
3,30.57118
4,28.64432
5,27.98462
6,25.27978


In [11]:
# df_sql_out

# R GLM Output

In [12]:
estimator  =  predict(model, dataset, type = "raw")
df_r_out = data.frame(estimator)
names(df_r_out) = c("Estimator")

df_r_out$KEY = seq.int(nrow(dataset))
head(df_r_out)


Estimator,KEY
30.06640,1
25.01567,2
30.57118,3
28.64432,4
27.98462,5
25.27978,6


# Compare R and SQL output

In [13]:
df_merge = merge(x = df_r_out, y = df_sql_out, by = "KEY", all = TRUE, , suffixes = c("_1","_2"))
head(df_merge)

KEY,Estimator_1,Estimator_2
1,30.06640,30.06640
2,25.01567,25.01567
3,30.57118,30.57118
4,28.64432,28.64432
5,27.98462,27.98462
6,25.27978,25.27978


In [14]:
df_merge$Error = df_merge$Estimator_1 - df_merge$Estimator_2
df_merge$AbsError = abs(df_merge$Error)
head(df_merge)


KEY,Estimator_1,Estimator_2,Error,AbsError
1,30.06640,30.06640,1.421085e-14,1.421085e-14
2,25.01567,25.01567,-7.105427e-15,7.105427e-15
3,30.57118,30.57118,3.552714e-15,3.552714e-15
4,28.64432,28.64432,0.000000e+00,0.000000e+00
5,27.98462,27.98462,3.552714e-15,3.552714e-15
6,25.27978,25.27978,3.552714e-15,3.552714e-15


In [15]:
df_merge_largest_errors = df_merge[df_merge$AbsError > 0.0001,]
df_merge_largest_errors

KEY,Estimator_1,Estimator_2,Error,AbsError


In [16]:
stopifnot(nrow(df_merge_largest_errors) == 0)

In [17]:
summary(df_sql_out)

      KEY          Estimator     
 Min.   :  1.0   Min.   :-4.209  
 1st Qu.:127.2   1st Qu.:17.490  
 Median :253.5   Median :22.167  
 Mean   :253.5   Mean   :22.533  
 3rd Qu.:379.8   3rd Qu.:27.423  
 Max.   :506.0   Max.   :44.596  

In [18]:
summary(df_r_out)

   Estimator           KEY       
 Min.   :-4.209   Min.   :  1.0  
 1st Qu.:17.490   1st Qu.:127.2  
 Median :22.167   Median :253.5  
 Mean   :22.533   Mean   :253.5  
 3rd Qu.:27.423   3rd Qu.:379.8  
 Max.   :44.596   Max.   :506.0  

In [19]:
summary(df_merge)

      KEY         Estimator_1      Estimator_2         Error           
 Min.   :  1.0   Min.   :-4.209   Min.   :-4.209   Min.   :-1.776e-14  
 1st Qu.:127.2   1st Qu.:17.490   1st Qu.:17.490   1st Qu.: 0.000e+00  
 Median :253.5   Median :22.167   Median :22.167   Median : 3.553e-15  
 Mean   :253.5   Mean   :22.533   Mean   :22.533   Mean   : 4.483e-15  
 3rd Qu.:379.8   3rd Qu.:27.423   3rd Qu.:27.423   3rd Qu.: 8.882e-15  
 Max.   :506.0   Max.   :44.596   Max.   :44.596   Max.   : 2.132e-14  
    AbsError        
 Min.   :0.000e+00  
 1st Qu.:3.553e-15  
 Median :7.105e-15  
 Mean   :6.575e-15  
 3rd Qu.:1.066e-14  
 Max.   :2.132e-14  